# Pricing Data Analyst Intern Task

### Import necessary libraries

In [121]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from datetime import datetime
import pandasql as psql
from scipy import stats
import numpy as np

### Read Excel Sheets

In [88]:
Orders = pd.read_excel("Data.xlsx", sheet_name='Sales_orders')
Cogs = pd.read_excel("Data.xlsx", sheet_name='Cost_of_Goods')
product = pd.read_excel("Data.xlsx", sheet_name='Product_sales_order')

## Clean Up Data

### Check for NAN

In [89]:
Orders_NAN = Orders[Orders.isna().any(axis=1)]
Orders_NAN

Orders_drop_NAN = Orders.dropna()
Orders_drop_NAN.head()


,ORDER_ID,DATE,DAY_NAME,Order_status,SALES
0,6387833,2023-10-01,Sun,Delivered,285.50
1,6385549,2023-10-01,Sun,Delivered,1512.25
2,6387475,2023-10-01,Sun,Delivered,197.50
3,6389331,2023-10-01,Sun,Delivered,67.50
4,6390122,2023-10-01,Sun,Delivered,118.00


### Check for duplicates

#### There is no duplicated rows

### Handle Outliers

In [ ]:
excel_file = 'path_to_your_excel_file.xlsx'
sheets_dict = pd.read_excel(excel_file, sheet_name=None)
def remove_outliers_iqr(df):
    """
    Removes outliers from a DataFrame using the IQR method.
    """
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    is_not_outlier = ~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR)))
    return df[is_not_outlier.all(axis=1)]

cleaned_sheets_dict = {sheet: remove_outliers_iqr(df) for sheet, df in sheets_dict.items()}

### Handle The Negative Values In The ['purchase_price']

In [90]:
#There is only one row that contains a negative value in the ['purchase_price]

#Add Flaged column to check if there are any negative values in the ['purchase_price']
Cogs['Flagged'] = Cogs['purchase_price'] < 0
Cogs[Cogs['Flagged']]


,product_id,purchase_price,selling_price,Flagged
7,361,-200.0,189.75,True


In [91]:
"""My recommendation, because it's only one row that it contains a negative value and it seems as it's an entry error;
so i will handle it by removing the negative sign"""

Cogs['purchase_price'] = Cogs['purchase_price'].abs()

### Working On Given Dataset1 (Sales_orders)

In [92]:
Orders

,ORDER_ID,DATE,DAY_NAME,Order_status,SALES
0,6387833,2023-10-01,Sun,Delivered,285.50
1,6385549,2023-10-01,Sun,Delivered,1512.25
2,6387475,2023-10-01,Sun,Delivered,197.50
3,6389331,2023-10-01,Sun,Delivered,67.50
4,6390122,2023-10-01,Sun,Delivered,118.00
...,...,...,...,...,...
17158,6509124,2023-10-21,Sat,Delivered,240.00
17159,6508933,2023-10-21,Sat,Delivered,266.25
17160,6503473,2023-10-21,Sat,Delivered,418.00
17161,6503157,2023-10-21,Sat,Delivered,746.25


#### Q1 → calculate total sales (delivered orders only) per day

In [93]:
filtered_sales = Orders[(Orders['Order_status'] == 'Delivered')]

grouped_sales = filtered_sales.groupby('DATE')['SALES'].sum()

grouped_sales

DATE
2023-10-01    3.708723e+05
2023-10-02    3.205579e+05
2023-10-03    4.498487e+05
2023-10-04    4.379692e+05
2023-10-05    2.256661e+05
2023-10-06   -9.771947e+06
2023-10-07    3.427147e+05
2023-10-08    3.936270e+05
2023-10-09    3.136172e+05
2023-10-10    2.758301e+05
2023-10-11    3.247948e+05
2023-10-12    2.340569e+05
2023-10-13    1.738548e+05
2023-10-14    2.777956e+05
2023-10-15    2.642121e+05
2023-10-16    2.909430e+05
2023-10-17    2.322276e+05
2023-10-18    2.983172e+05
2023-10-19    2.543309e+05
2023-10-20    2.035265e+05
2023-10-21    2.818321e+05
Name: SALES, dtype: float64

#### Q2 → Given the 3 weeks of data from sales orders predict the 4th-week (from 22nd to 28th of October) total sales (delivered only).

In [112]:
# Ensure 'date' column is in datetime format
Orders['DATE'] = pd.to_datetime(Orders['DATE'])

# Filter to include only the relevant dates for the first 3 weeks
historical_data = Orders[Orders['DATE'] < '2023-10-22'].copy()

# Check for NaN values in 'SALES' column
nan_indices = historical_data['SALES'].index[historical_data['SALES'].isna()]
print(f"NaN values found in historical_data['SALES'] at indices: {nan_indices}")

# Drop rows with NaN values from historical_data
historical_data.drop(index=nan_indices, inplace=True)

# Prepare training data
X_train = historical_data['DATE'].apply(lambda x: x.toordinal()).values.reshape(-1, 1)
y_train = historical_data['SALES']

# Fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict sales for the 4th week
week_4_start = datetime.strptime('2023-10-22', '%Y-%m-%d')
week_4_end = datetime.strptime('2023-10-28', '%Y-%m-%d')
X_test = pd.date_range(start=week_4_start, end=week_4_end, freq='D')
X_test_ordinal = X_test.to_series().apply(lambda x: x.toordinal()).values.reshape(-1, 1)

predicted_sales = model.predict(X_test_ordinal)

# Print or use predicted_sales
print("Predicted sales for the 4th week:")
for date, sales in zip(X_test, predicted_sales):
    print(f"{date.date()}: {sales}")


NaN values found in historical_data['SALES'] at indices: Index([29], dtype='int64')
Predicted sales for the 4th week:
2023-10-22: 583.9579236581922
2023-10-23: 650.2610296010971
2023-10-24: 716.564135544002
2023-10-25: 782.8672414794564
2023-10-26: 849.1703474223614
2023-10-27: 915.4734533652663
2023-10-28: 981.7765593081713


#### Q3 → Based on your prediction in the previous question, what will be the contribution of Wednesday and Friday in the 4th week (from 22nd to 28th of October).

In [114]:
# Create a DataFrame for the predicted sales
predicted_sales = pd.DataFrame({'DATE': X_test, 'PREDICTED_SALES': predicted_sales})

# Calculate the contribution of Wednesday (2023-10-25) and Friday (2023-10-27)
wednesday_sales = predicted_sales[predicted_sales['DATE'] == '2023-10-25']['PREDICTED_SALES'].values[0]
friday_sales = predicted_sales[predicted_sales['DATE'] == '2023-10-27']['PREDICTED_SALES'].values[0]
total_sales = predicted_sales['PREDICTED_SALES'].sum()

# Contribution of Wednesday and Friday
wednesday_contribution = (wednesday_sales / total_sales) * 100
friday_contribution = (friday_sales / total_sales) * 100

print(f"Wednesday's predicted sales: {wednesday_sales}")
print(f"Friday's predicted sales: {friday_sales}")
print(f"Total predicted sales for the week: {total_sales}")

print(f"Wednesday's contribution: {wednesday_contribution:.2f}%")
print(f"Friday's contribution: {friday_contribution:.2f}%")

Wednesday's predicted sales: 782.8672414794564
Friday's predicted sales: 915.4734533652663
Total predicted sales for the week: 5480.070690378547
Wednesday's contribution: 14.29%
Friday's contribution: 16.71%


#### Q4 → Based on the 4 weeks of data, Which week has the highest sales and which day is usually the highest per week in terms of sales (delivered only)

In [115]:
# Add 'WEEK' and 'DAY_OF_WEEK' columns
Orders['WEEK'] = Orders['DATE'].dt.isocalendar().week
Orders['DAY_OF_WEEK'] = Orders['DATE'].dt.day_name()

# Aggregate sales by week
weekly_sales = Orders.groupby('WEEK')['SALES'].sum().reset_index()
highest_sales_week = weekly_sales.loc[weekly_sales['SALES'].idxmax()]

# Aggregate sales by day of the week
daily_sales = Orders.groupby('DAY_OF_WEEK')['SALES'].sum().reset_index()
highest_sales_day = daily_sales.loc[daily_sales['SALES'].idxmax()]

# Output the results
print("Weekly Sales Summary:")
print(weekly_sales)
print("\nWeek with Highest Sales:")
print(f"Week: {highest_sales_week['WEEK']}, Sales: {highest_sales_week['SALES']}")

print("\nDaily Sales Summary:")
print(daily_sales)
print("\nDay with Highest Sales:")
print(f"Day: {highest_sales_day['DAY_OF_WEEK']}, Sales: {highest_sales_day['SALES']}")

Weekly Sales Summary:
   WEEK         SALES
0    39  4.036020e+05
1    40 -7.376878e+06
2    41  2.083224e+06
3    42  1.699983e+06

Week with Highest Sales:
Week: 41.0, Sales: 2083223.9403

Daily Sales Summary:
  DAY_OF_WEEK         SALES
0      Friday -9.345287e+06
1      Monday  1.024110e+06
2    Saturday  9.814411e+05
3      Sunday  1.115708e+06
4    Thursday  7.871651e+05
5     Tuesday  1.060926e+06
6   Wednesday  1.185868e+06

Day with Highest Sales:
Day: Wednesday, Sales: 1185868.2535


### Working on Given Dataset2 (Cost_of_Goods) and Dataset3 (Product_sales_order)

In [62]:
#Dataset2 (Cost_of_Goods)
Cogs.head()

,product_id,purchase_price,selling_price,Margin_Percent,Gross_Profit,Flagged
0,415,270.56,281.75,3.971606,11.19,False
1,3495,20.01,21.75,8.000000,1.74,False
2,152,205.08,212.25,3.378092,7.17,False
3,72,508.79,540.75,5.910310,31.96,False
4,974,134.01,144.25,7.098787,10.24,False


In [63]:
#Dataset3 (Product_sales_order)
product.head()

,PRODUCT_SALES_ORDER_ID,SALES_ORDER_ID,PRODUCT_ID,SALES
0,68097611,6460123,2438,390.00
1,68009163,6448262,1087,122.25
2,68097089,6460054,142,166.32
3,68060918,6455556,415,226.50
4,68144928,6466586,361,336.00


#### Q5 → Calculate the margin percentage per product.

In [64]:
#Margin Percentage equal ( (Selling Price - Cost Price) / Selling Price ) * 100

#Cost Price is the same to purchase price

#Create a Margin Percentage Column
Cogs['Margin_Percent'] = ((Cogs['selling_price'] - Cogs['purchase_price']) / Cogs['selling_price']) * 100

Cogs.head()

,product_id,purchase_price,selling_price,Margin_Percent,Gross_Profit,Flagged
0,415,270.56,281.75,3.971606,11.19,False
1,3495,20.01,21.75,8.000000,1.74,False
2,152,205.08,212.25,3.378092,7.17,False
3,72,508.79,540.75,5.910310,31.96,False
4,974,134.01,144.25,7.098787,10.24,False


#### Q6 → Calculate the gross profit per product.

In [65]:
#Gross Profit Equal (Selling Price - Cost Price)

#Create a Column For Gross Profit
Cogs['Gross_Profit'] = Cogs['selling_price'] - Cogs['purchase_price']

Cogs.head()

,product_id,purchase_price,selling_price,Margin_Percent,Gross_Profit,Flagged
0,415,270.56,281.75,3.971606,11.19,False
1,3495,20.01,21.75,8.000000,1.74,False
2,152,205.08,212.25,3.378092,7.17,False
3,72,508.79,540.75,5.910310,31.96,False
4,974,134.01,144.25,7.098787,10.24,False


#### Q7 → What are the top 3 and bottom 3 products in terms of gross profit.

In [66]:
#Sort the data by [Gross_Profit]
Cogs_Sorted = Cogs.sort_values(by='Gross_Profit', ascending=False)

In [67]:
#Get The Top 3 Products in Terms of Gross Profit
Cogs_Sorted.head(3)

,product_id,purchase_price,selling_price,Margin_Percent,Gross_Profit,Flagged
6,3900,501.99,557.75,9.997311,55.76,False
3,72,508.79,540.75,5.910310,31.96,False
24,9289,388.80,408.25,4.764238,19.45,False


In [68]:
#Get The Bottom 3 Products in Terms of Gross Profit
Cogs_Sorted.tail(3)

,product_id,purchase_price,selling_price,Margin_Percent,Gross_Profit,Flagged
1,3495,2.001000e+01,21.75,8.000000e+00,1.74,False
7,361,2.000000e+02,189.75,-5.401845e+00,-10.25,False
15,168,1.000000e+08,68.50,-1.459853e+08,-99999931.50,False


#### Q8 → State your recommendations for how we could further increase our gross profit.

### Recommendations!

1. Increase Sales Volume of High Gross Profit Products:
Focus marketing and sales efforts on products with the highest gross profit. This can be done through targeted promotions, bundling products, or increasing their visibility.

2. Price Optimization:
Focus On products with lower margins but high sales volume like ptoduct(1222).

3. Cost Reduction:
Negotiate better purchase prices with suppliers for products with lower margins. Even a small reduction in purchase price can significantly increase the margin and gross profit.

### SQL Section 

#### Q9 → If the 3 datasets (Sales_orders, Cost_of_Goods & Product_sales_order) were tables in database, write a simple SQL query to create 1 table that has the total number of unique delivered products and the total number of unique canceled products per day for the first week and create a flag that is equal to 1 if that day had more than than 5 unique products delivered and 0 else wise. Also make sure your answer doesn't contain fully duplicated rows

### SQL Query Commands

WITH CombinedData AS (
    SELECT 
        so.ORDER_ID, 
        so.DATE,
        pso.product_id,
        so.Order_status
    FROM 
        Sales_orders AS so
    JOIN 
        Product_sales_order AS pso
    ON 
        pso.SALES_ORDER_ID = so.ORDER_ID
    WHERE 
        so.DATE BETWEEN '2023-10-01' AND '2023-10-07'
),
UniqueDelivered AS (
    SELECT 
        DATE,
        COUNT(DISTINCT product_id) AS unique_delivered
    FROM 
        CombinedData
    WHERE 
        Order_status = 'Delivered'
    GROUP BY 
        DATE
),
UniqueCanceled AS (
    SELECT 
        DATE,
        COUNT(DISTINCT product_id) AS unique_canceled
    FROM 
        CombinedData
    WHERE 
        Order_status = 'Canceled'
    GROUP BY 
        DATE
)
SELECT 
    ud.DATE,
    ud.unique_delivered,
    uc.unique_canceled,
    CASE 
        WHEN ud.unique_delivered > 5 THEN 1 
        ELSE 0 
    END AS flag
FROM 
    UniqueDelivered AS ud
LEFT JOIN 
    UniqueCanceled AS uc
ON 
    ud.DATE = uc.DATE
ORDER BY 
    ud.DATE;
